In [1]:
import chromadb
from google import genai
from chromadb.api.types import EmbeddingFunction


In [2]:
CHROMA_PATH = "./REGULATIONS/chroma_db"
COLLECTION_NAME = "regulations"


In [3]:
client_gemini = genai.Client()

class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts):
        if isinstance(texts, str):
            texts = [texts]

        response = client_gemini.models.embed_content(
            model="models/text-embedding-004",
            contents=texts
        )

        return [e.values for e in response.embeddings]

    def name(self):
        return "gemini-text-embedding-004"


In [4]:
chroma_client = chromadb.PersistentClient(
    path=CHROMA_PATH
)

embedding_fn = GeminiEmbeddingFunction()

collection = chroma_client.get_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn
)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21628\2526141599.py:5: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_fn = GeminiEmbeddingFunction()


In [6]:
query_text = "How sensitive is persons' sexual orientation?"

results = collection.query(
    query_texts=[query_text],
    n_results=5,
    include=["documents", "metadatas", "distances"]
)

# Extract documents and metadata
texts = results['documents'][0]   # list of text chunks
metadatas = results['metadatas'][0]  # corresponding metadata

# Combine text with metadata
text_with_metadata = []
for text, meta in zip(texts, metadatas):
    text_with_metadata.append({
        'text': text,
        'metadata': meta
    })

# Print
# for item in text_with_metadata:
#     print("Text:", item['text'][:200], "...")  # first 200 chars
#     print("Metadata:", item['metadata'])
#     print("------")

text_with_metadata
# print("\n🔍 Top Results:\n")

# for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
#     print(doc[:400])
#     print("📄 Source:", meta["source"], "| Page:", meta["page"])
#     print("-" * 80)

[{'text': "Processing of personal data revealing racial or ethnic or igin, political opinions, religious or philosophical beliefs, or trade union membership, and the processing of genetic data, biometr ic data f or the pur pose of uniquely identifying a natural person, data concer ning health or data concer ning a natural person's sex life or sexual or ientation shall be prohibit ed. 2.",
  'metadata': {'source': 'GDPR.pdf', 'page': 38}},
 {'text': '(75) The r isk to the r ights and freedoms of natural persons, of var ying lik elihood and sever ity , ma y result from personal data processing whic h could lead to phy sical, mater ial or non-materi al damag e, in par ticular: where the processing ma y give r ise to discr imination, identity thef t or fraud, financial loss, damage to the reputation, loss of confidentiality of personal data prot ected by profess ional secrecy , unauthor ised reversal of pseudon ymisation, or an y other significant economic or social disadvantag e; where da